Code used from this link:

https://medium.com/technonerds/using-fastais-ulmfit-to-make-a-state-of-the-art-multi-label-text-classifier-bf54e2943e83

GitHub Link: 
https://github.com/aditya10/ULMFiT-fastai-text-classifier/blob/master/ULMFiT_tutorial.ipynb



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from fastai.text import *
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

###Model set-up

In [0]:
# load train_cleaned.csv

filename = "/content/gdrive/My Drive/train_dataset/train_tabular_c4.csv"
train = pd.read_csv(filename)

In [0]:
train_language = pd.DataFrame()

train_language['target'] = train['AdoptionSpeed']
train_language['text'] = train['Description']

In [0]:
train_language['target'].replace(to_replace=0, value=1, inplace=True)

In [0]:
train_language.head()

In [0]:
#create train and test split
df_trn, df_val = train_test_split(train_language, stratify = train_language['target'], test_size = 0.3, random_state=42)
df_trn.shape, df_val.shape

###Pre-processing language model

In [9]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [10]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "", vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
# check out data
data_clas.show_batch()

In [0]:
data_clas.save("/content/gdrive/My Drive/language_models/databunch/TextClasDataBunch.pkl")

In [0]:
data_lm.save("/content/gdrive/My Drive/language_models/databunch/TextLMDataBunch.pkl")

### Language model

In [0]:
# train language model
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
# find the optimal learning rate to train our language model on
learn.lr_find()
learn.recorder.plot(suggestion=True)
min_grad_lr = learn.recorder.min_grad_lr

In [0]:
# let’s use this learning rate to train the language model
learn.fit_one_cycle(10, min_grad_lr)

In [0]:
# unfreezing weights and training the rest of the NN
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

Our language model is able to predict the next word with 40% accuracy. This is not bad. 

In [0]:
# save the language model encoder so that we can load it later in our classifier
learn.save_encoder('/content/gdrive/My Drive/language_models/models/ft_enc')

### Language classifier

In [0]:
# implement weighted cohen's kappa metric
kappa = KappaScore()
kappa.weights = "quadratic"

In [0]:
#set-up classifier
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, kappa])
learn.load_encoder('/content/gdrive/My Drive/language_models/models/ft_enc')

In [0]:
#train classifier for 10 epochs
learn.fit_one_cycle(10)

In [0]:
#save model weights
learn.save('/content/gdrive/My Drive/language_models/models/language_classifier')

In [0]:
#inspect losses.
learn.recorder.plot_losses()

In [0]:
# unfreezing weights and training the rest of the NN
learn.freeze_to(-2)
learn.fit_one_cycle(4, slice(5e-3, 2e-3), moms=(0.8,0.7))

In [0]:
#export language classifier for prediction
learn.export(file = "/content/gdrive/My Drive/language_models/nlp_export.pkl")

###Confusion matrix

In [0]:
# create confusion matrix
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)
interp.plot_confusion_matrix()